# 思路1--数据集切分（不叠加）

# 思路1 法1 普通求占比

In [ ]:
import pandas as pd
import numpy as np
import shap
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt

# 定义MAPE函数
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# 加载和预处理数据
file_path = r'F:\8--BJTU大三\大三下\商务智能 选修\团队作业\ubaar-competition\train50k_processed.csv'
df = pd.read_csv(file_path)
df = df.iloc[:5000]

# 处理'price'列的异常值
q99 = df['price'].quantile(0.99)
df = df[df['price'] <= q99]

# 拆分数据
X = df.drop(columns=['price'])
y = df['price']

# 计算'taxiDurationMin'的四分位数
quartiles = df['taxiDurationMin'].quantile([0.25, 0.5, 0.75])

# 按四分位数拆分数据集
X_0_25 = X[X['taxiDurationMin'] <= quartiles[0.25]]
y_0_25 = y[X['taxiDurationMin'] <= quartiles[0.25]]

X_25_50 = X[(X['taxiDurationMin'] > quartiles[0.25]) & (X['taxiDurationMin'] <= quartiles[0.5])]
y_25_50 = y[(X['taxiDurationMin'] > quartiles[0.25]) & (X['taxiDurationMin'] <= quartiles[0.5])]

X_50_75 = X[(X['taxiDurationMin'] > quartiles[0.5]) & (X['taxiDurationMin'] <= quartiles[0.75])]
y_50_75 = y[(X['taxiDurationMin'] > quartiles[0.5]) & (X['taxiDurationMin'] <= quartiles[0.75])]

X_75_100 = X[X['taxiDurationMin'] > quartiles[0.75]]
y_75_100 = y[X['taxiDurationMin'] > quartiles[0.75]]

# 训练模型并计算整体数据集的Shapley值
model = RandomForestRegressor()
model.fit(X, y)

explainer = shap.Explainer(model, X)
overall_shap_values = explainer(X, check_additivity=False)

# 计算每个数据子集的Shapley值
sample_groups = [(X_0_25, y_0_25), (X_25_50, y_25_50), (X_50_75, y_50_75), (X_75_100, y_75_100)]
sample_shap_values = []

for X_sample, y_sample in sample_groups:
    explainer_sample = shap.Explainer(model, X_sample)
    sample_shap_values.append(explainer_sample(X_sample, check_additivity=False))

# 计算数据贡献率
# 使用Shapley值计算每个样本的贡献
overall_mean_contribution = np.mean(np.abs(overall_shap_values.values), axis=0)

# 计算每个样本集的贡献率
epsilon = 1e-10  # 小常数，避免除以零
contribution_rates = []
for shap_values in sample_shap_values:
    sample_mean_contribution = np.mean(np.abs(shap_values.values), axis=0)
    contribution_rate = np.mean(sample_mean_contribution / (overall_mean_contribution + epsilon))
    contribution_rates.append(contribution_rate)

# 输出贡献率
for i, rate in enumerate(contribution_rates):
    print(f'Sample group {i + 1} contribution rate: {rate}')

# 绘制每个数据子集的Shapley值图
subset_titles = ['0-25th Percentile', '25-50th Percentile', '50-75th Percentile', '75-100th Percentile']
for i, shap_values in enumerate(sample_shap_values):
    plt.figure(figsize=(20, 10))
    shap.summary_plot(shap_values, sample_groups[i][0])
    plt.title(f'Sample group {i + 1} SHAP values ({subset_titles[i]})')
    plt.show()


# 思路1 法2 合作博弈论shapley思想

In [ ]:
import pandas as pd
import numpy as np
import shap
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_percentage_error
import matplotlib.pyplot as plt

# 定义MAPE函数
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# 加载和预处理数据
file_path = r'F:\8--BJTU大三\大三下\商务智能 选修\团队作业\ubaar-competition\train50k_processed.csv'
df = pd.read_csv(file_path)
df = df.iloc[:5000]

# 处理'price'列的异常值
q99 = df['price'].quantile(0.99)
df = df[df['price'] <= q99]

# 拆分数据
X = df.drop(columns=['price'])
y = df['price']

# 计算'taxiDurationMin'的四分位数
quartiles = df['taxiDurationMin'].quantile([0.25, 0.5, 0.75])

# 按四分位数拆分数据集
X_0_25 = X[X['taxiDurationMin'] <= quartiles[0.25]]
y_0_25 = y[X['taxiDurationMin'] <= quartiles[0.25]]

X_25_50 = X[(X['taxiDurationMin'] > quartiles[0.25]) & (X['taxiDurationMin'] <= quartiles[0.5])]
y_25_50 = y[(X['taxiDurationMin'] > quartiles[0.25]) & (X['taxiDurationMin'] <= quartiles[0.5])]

X_50_75 = X[(X['taxiDurationMin'] > quartiles[0.5]) & (X['taxiDurationMin'] <= quartiles[0.75])]
y_50_75 = y[(X['taxiDurationMin'] > quartiles[0.5]) & (X['taxiDurationMin'] <= quartiles[0.75])]

X_75_100 = X[X['taxiDurationMin'] > quartiles[0.75]]
y_75_100 = y[X['taxiDurationMin'] > quartiles[0.75]]

# 定义子集
sample_groups = [(X_0_25, y_0_25, '0-25th Percentile'), 
                 (X_25_50, y_25_50, '25-50th Percentile'), 
                 (X_50_75, y_50_75, '50-75th Percentile'), 
                 (X_75_100, y_75_100, '75-100th Percentile')]

# 训练模型并计算整体数据集的MAPE
model = RandomForestRegressor()
model.fit(X, y)
y_pred = model.predict(X)
overall_mape = mean_absolute_percentage_error(y, y_pred)

# 计算每个子集的边际贡献
marginal_contributions = []
for X_sample, y_sample, label in sample_groups:
    # 移除当前子集
    X_reduced = X.drop(X_sample.index)
    y_reduced = y.drop(y_sample.index)
    
    # 训练模型并计算MAPE
    model_reduced = RandomForestRegressor()
    model_reduced.fit(X_reduced, y_reduced)
    y_reduced_pred = model_reduced.predict(X_reduced)
    reduced_mape = mean_absolute_percentage_error(y_reduced, y_reduced_pred)
    
    # 计算边际贡献
    marginal_contribution = overall_mape - reduced_mape
    marginal_contributions.append((label, marginal_contribution))

# 输出边际贡献
for label, contribution in marginal_contributions:
    print(f'{label} marginal contribution: {contribution}')

# 训练模型并计算每个数据子集的Shapley值
explainer = shap.Explainer(model, X)
overall_shap_values = explainer(X, check_additivity=False)
sample_shap_values = []

for X_sample, y_sample, label in sample_groups:
    explainer_sample = shap.Explainer(model, X_sample)
    sample_shap_values.append((X_sample, label, explainer_sample(X_sample, check_additivity=False)))

# 绘制每个数据子集的Shapley值图
for X_sample, label, shap_values in sample_shap_values:
    plt.figure(figsize=(20, 10))
    shap.summary_plot(shap_values, X_sample, show=False)
    plt.title(f'SHAP values for {label}')
    plt.show()


# 以上均为早期错误思想，但可能也有点小对，你们看着理解下，还用不用的上

# 正确思想

In [1]:
import pandas as pd
import numpy as np
from itertools import permutations, combinations
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_percentage_error, r2_score, mean_squared_error
import matplotlib.pyplot as plt

# 加载和预处理数据
file_path = r'F:\8--BJTU大三\大三下\商务智能 选修\团队作业\ubaar-competition\train50k_processed.csv'
df = pd.read_csv(file_path)
df = df.iloc[:5000]

# 处理'price'列的异常值
q99 = df['price'].quantile(0.99)
df = df[df['price'] <= q99]

# 拆分数据
X = df.drop(columns=['price'])
y = df['price']

# 将数据集按10%的比例分成10份
n_splits = 10
split_size = len(X) // n_splits
splits = [(X.iloc[i*split_size:(i+1)*split_size], y.iloc[i*split_size:(i+1)*split_size]) for i in range(n_splits)]

# 初始模型训练
model = RandomForestRegressor()
model.fit(X, y)
initial_y_pred = model.predict(X)
initial_mape = mean_absolute_percentage_error(y, initial_y_pred)
initial_r2 = r2_score(y, initial_y_pred)
initial_mse = mean_squared_error(y, initial_y_pred)

# Shapley值计算
shapley_values_mape = np.zeros(n_splits)
shapley_values_r2 = np.zeros(n_splits)
shapley_values_mse = np.zeros(n_splits)

for perm in permutations(range(n_splits)):
    X_train = pd.DataFrame()
    y_train = pd.Series(dtype=y.dtype)
    prev_mape = initial_mape
    prev_r2 = initial_r2
    prev_mse = initial_mse
    for i in perm:
        X_train = pd.concat([X_train, splits[i][0]])
        y_train = pd.concat([y_train, splits[i][1]])
        
        model.fit(X_train, y_train)
        y_pred = model.predict(X)
        
        mape = mean_absolute_percentage_error(y, y_pred)
        r2 = r2_score(y, y_pred)
        mse = mean_squared_error(y, y_pred)
        
        shapley_values_mape[i] += prev_mape - mape
        shapley_values_r2[i] += r2 - prev_r2
        shapley_values_mse[i] += prev_mse - mse
        
        prev_mape = mape
        prev_r2 = r2
        prev_mse = mse

shapley_values_mape /= len(list(permutations(range(n_splits))))
shapley_values_r2 /= len(list(permutations(range(n_splits))))
shapley_values_mse /= len(list(permutations(range(n_splits))))

# 输出Shapley值
for i in range(n_splits):
    print(f'Shapley value for subset {i+1}:')
    print(f'ΔMAPE: {shapley_values_mape[i]}')
    print(f'ΔR2: {shapley_values_r2[i]}')
    print(f'ΔMSE: {shapley_values_mse[i]}')

# 绘制变化曲线图
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.plot(range(1, n_splits+1), shapley_values_mape, marker='o')
plt.title('Shapley ΔMAPE')
plt.xlabel('Subset')
plt.ylabel('ΔMAPE')

plt.subplot(1, 3, 2)
plt.plot(range(1, n_splits+1), shapley_values_r2, marker='o')
plt.title('Shapley ΔR2')
plt.xlabel('Subset')
plt.ylabel('ΔR2')

plt.subplot(1, 3, 3)
plt.plot(range(1, n_splits+1), shapley_values_mse, marker='o')
plt.title('Shapley ΔMSE')
plt.xlabel('Subset')
plt.ylabel('ΔMSE')

plt.tight_layout()
plt.show()


C:\Users\31912\AppData\Roaming\Python\Python39\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\31912\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (
C:\Users\31912\AppData\Local\Temp\ipykernel_6356\1632260225.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pan

KeyboardInterrupt: 